In [27]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
import json
import gradio as gr
from duckduckgo_search import DDGS

In [25]:
load_dotenv(override=True)

openai_api = os.getenv("OPENAI_API_KEY")
serper_api = os.getenv("X-API-KEY")

if openai_api:
    print(f"Exists and starts with {openai_api[:10]}")

if serper_api:
    print(f"Exists and starts with {serper_api[:10]}")

Exists and starts with sk-proj-61
Exists and starts with db85bd4af6


In [19]:
openai = OpenAI()
MODEL = "gpt-5-mini"

In [17]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [8]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(city):
    print(f"Tool called for {city.lower()}")
    price = ticket_prices.get(city.lower(), "Unknown city")
    return f"The price for {city.lower()} is {price}"

In [10]:
get_ticket_price('PARIS')

Tool called for paris


'The price for paris is $899'

In [13]:
price_function = {
    "type": "function",
    "function": {
        "name": "get_ticket_price",
        "description": "Returns the cost of a round-trip flight to a specific city.",
        "parameters": {
            "type": "object",
            "properties": {
                "destination_city": {
                    "type": "string",
                    "description": "The city name, e.g., 'London'"
                },
            },
            "required": ["destination_city"],
            "additionalProperties": False
        }
    }
}

In [45]:
def search_internet(query):
    url = "https://google.serper.dev/search"
    payload = json.dumps({"q": query, "gl": "us", "hl": "en"})
    headers = {
        'X-API-KEY': serper_api,
        'Content-Type': 'application/json'
    }

    response = requests.post(url, headers=headers, data=payload)
    results = response.json().get('organic', [])
    
    snippets = [r.get('snippet', '') for r in results[:3]]
    return "\n\n".join(snippets) if snippets else "No results found."

In [46]:
search_internet("what is the weather like in Tirana?")

'Partly cloudy skies early then becoming cloudy with periods of rain late. Low 49F. Winds SSE at 10 to 15 mph. Chance of rain 90%. Rainfall may reach one inch.\n\nHourly Weather · 1 AM 43°. rain drop 11% · 2 AM 42°. rain drop 11% · 3 AM 42°. rain drop 11% · 4 AM 41°. rain drop 11% · 5 AM 40°. rain drop 11% · 6 AM 38°.\n\n... weather forecast for Tirana ... Temperature feels like 13°55°. High chance of precipitation. Light winds from the south-south-east. 02:00. ,. Heavy Rain.'

In [47]:
search_tool = {
    "type": "function",
    "function": {
        "name": "search_internet",
        "description": "Search the web for current events or facts.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string", 
                    "description": "The search query"
                    },
            },
            "required": ["query"],
            "additionalProperties": False
        }
    }
}

In [48]:
tools = [price_function, search_tool]
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Returns the cost of a round-trip flight to a specific city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': "The city name, e.g., 'London'"}},
    'required': ['destination_city'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'search_internet',
   'description': 'Search the web for current events or facts.',
   'parameters': {'type': 'object',
    'properties': {'query': {'type': 'string',
      'description': 'The search query'}},
    'required': ['query'],
    'additionalProperties': False}}}]

In [49]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        
        if function_name == "get_ticket_price":
            city = arguments.get("destination_city")
            result = get_ticket_price(city)
        elif function_name == "search_internet":
            query = arguments.get("query")
            result = search_internet(query)
        else:
            result = "Error: Tool not found."

        responses.append({
            "role": "tool",
            "content": str(result),
            "tool_call_id": tool_call.id
        })
        
    return responses

In [50]:
def chat(message, history):
    history = [{"role":h[ "role"], "content":h ["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    return response.choices[0].message.content

In [51]:
gr.ChatInterface(fn=chat, type="messages", flagging_mode="never").launch()

/opt/homebrew/Caskroom/miniforge/base/envs/llms/lib/python3.11/site-packages/gradio/http_server.py:120: ResourceWarning: unclosed <socket.socket fd=109, family=2, type=1, proto=0, laddr=('0.0.0.0', 0)>
  s = socket.socket()
/opt/homebrew/Caskroom/miniforge/base/envs/llms/lib/python3.11/site-packages/gradio/http_server.py:120: ResourceWarning: unclosed <socket.socket fd=110, family=2, type=1, proto=0, laddr=('0.0.0.0', 0)>
  s = socket.socket()


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


/opt/homebrew/Caskroom/miniforge/base/envs/llms/lib/python3.11/site-packages/gradio/routes.py:1334: DeprecationWarning: 'HTTP_422_UNPROCESSABLE_ENTITY' is deprecated. Use 'HTTP_422_UNPROCESSABLE_CONTENT' instead.
  return await queue_join_helper(body, request, username)
/opt/homebrew/Caskroom/miniforge/base/envs/llms/lib/python3.11/site-packages/gradio/routes.py:1334: DeprecationWarning: 'HTTP_422_UNPROCESSABLE_ENTITY' is deprecated. Use 'HTTP_422_UNPROCESSABLE_CONTENT' instead.
  return await queue_join_helper(body, request, username)
/opt/homebrew/Caskroom/miniforge/base/envs/llms/lib/python3.11/site-packages/gradio/routes.py:1334: DeprecationWarning: 'HTTP_422_UNPROCESSABLE_ENTITY' is deprecated. Use 'HTTP_422_UNPROCESSABLE_CONTENT' instead.
  return await queue_join_helper(body, request, username)


Tool called for tokyo


/opt/homebrew/Caskroom/miniforge/base/envs/llms/lib/python3.11/site-packages/gradio/routes.py:1334: DeprecationWarning: 'HTTP_422_UNPROCESSABLE_ENTITY' is deprecated. Use 'HTTP_422_UNPROCESSABLE_CONTENT' instead.
  return await queue_join_helper(body, request, username)
/opt/homebrew/Caskroom/miniforge/base/envs/llms/lib/python3.11/site-packages/gradio/routes.py:1334: DeprecationWarning: 'HTTP_422_UNPROCESSABLE_ENTITY' is deprecated. Use 'HTTP_422_UNPROCESSABLE_CONTENT' instead.
  return await queue_join_helper(body, request, username)


...

/opt/homebrew/Caskroom/miniforge/base/envs/llms/lib/python3.11/site-packages/gradio/routes.py:1334: DeprecationWarning: 'HTTP_422_UNPROCESSABLE_ENTITY' is deprecated. Use 'HTTP_422_UNPROCESSABLE_CONTENT' instead.
  return await queue_join_helper(body, request, username)
/opt/homebrew/Caskroom/miniforge/base/envs/llms/lib/python3.11/site-packages/gradio/routes.py:1334: DeprecationWarning: 'HTTP_422_UNPROCESSABLE_ENTITY' is deprecated. Use 'HTTP_422_UNPROCESSABLE_CONTENT' instead.
  return await queue_join_helper(body, request, username)


Tool called for albania


/opt/homebrew/Caskroom/miniforge/base/envs/llms/lib/python3.11/site-packages/gradio/routes.py:1334: DeprecationWarning: 'HTTP_422_UNPROCESSABLE_ENTITY' is deprecated. Use 'HTTP_422_UNPROCESSABLE_CONTENT' instead.
  return await queue_join_helper(body, request, username)
/opt/homebrew/Caskroom/miniforge/base/envs/llms/lib/python3.11/site-packages/gradio/routes.py:1334: DeprecationWarning: 'HTTP_422_UNPROCESSABLE_ENTITY' is deprecated. Use 'HTTP_422_UNPROCESSABLE_CONTENT' instead.
  return await queue_join_helper(body, request, username)


Tool called for tirana


/opt/homebrew/Caskroom/miniforge/base/envs/llms/lib/python3.11/site-packages/gradio/routes.py:1334: DeprecationWarning: 'HTTP_422_UNPROCESSABLE_ENTITY' is deprecated. Use 'HTTP_422_UNPROCESSABLE_CONTENT' instead.
  return await queue_join_helper(body, request, username)
/opt/homebrew/Caskroom/miniforge/base/envs/llms/lib/python3.11/site-packages/gradio/routes.py:1334: DeprecationWarning: 'HTTP_422_UNPROCESSABLE_ENTITY' is deprecated. Use 'HTTP_422_UNPROCESSABLE_CONTENT' instead.
  return await queue_join_helper(body, request, username)
/opt/homebrew/Caskroom/miniforge/base/envs/llms/lib/python3.11/site-packages/gradio/routes.py:1334: DeprecationWarning: 'HTTP_422_UNPROCESSABLE_ENTITY' is deprecated. Use 'HTTP_422_UNPROCESSABLE_CONTENT' instead.
  return await queue_join_helper(body, request, username)
/opt/homebrew/Caskroom/miniforge/base/envs/llms/lib/python3.11/site-packages/gradio/routes.py:1334: DeprecationWarning: 'HTTP_422_UNPROCESSABLE_ENTITY' is deprecated. Use 'HTTP_422_UNPROC